In [32]:
from langchain_community.document_loaders import CSVLoader
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

import os
import sys

sys.path.append('..')  # add parent dir to sys.path
import utils

In [6]:
# load csv-docs
loader = CSVLoader(
    file_path='datasets/lenta-ru-news-short.csv',
    csv_args={'fieldnames': ['Заголовок', 'Содержание', 'Тема', 'Теги', 'Дата']}
)

docs = loader.load()
docs[:3]

[Document(metadata={'source': 'datasets/lenta-ru-news-short.csv', 'row': 0}, page_content='Заголовок: «Тольяттиазот» нацелился на\xa0смену лидера\nСодержание: Акционерам крупнейшего российского производителя аммиака — химкомбината «Тольяттиазот» в скором времени будет предложено избрать новый состав совета директоров, но уже без председателя Сергея Махлая, возглавлявшего совет с 2011 года. Об этом сообщил «Интерфаксу» источник, близкий к совету директоров предприятия. По его словам, Махлай направил в адрес компании отказ от выдвижения своей кандидатуры в новый состав совета директоров. При этом подготовка к проведению собрания акционеров уже началась. В частности, была определена повестка дня собрания. По мнению генерального директора «Центра политической информации» Алексея Мухина, принятое Махлаем решение пойдет на пользу предприятию и лишний раз демонстрирует его способность находить неожиданные и эффективные управленческие стратегии. «Это волевое решение. Выход Махлая, скорее всего

In [7]:
# embeddings model
model_kwargs = {'device': 'mps'} # use GPU
encode_kwargs = {'normalize_embeddings': True}

embeddings = HuggingFaceEmbeddings(
    model_name='cointegrated/LaBSE-en-ru',
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs,
)

In [8]:
%%time
# create or load vector store

if not os.path.exists('faiss-index'):
    db = FAISS.from_documents(
        documents=docs,
        embedding=embeddings,
    )
    db.save_local(
        folder_path='faiss-index',
        index_name='index'
    )

else:
    db = FAISS.load_local(
        folder_path='faiss-index',
        index_name='index',
        embeddings=embeddings,
        allow_dangerous_deserialization=True,
    )

CPU times: user 66.2 ms, sys: 32.7 ms, total: 98.9 ms
Wall time: 124 ms


In [9]:
retriever = db.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={'k': 3, 'score_threshold': 0.2},
)

In [10]:
query = 'Когда будет праздноваться день рождения Лермонтова?'

rel_docs = retriever.invoke(query)

for idx, doc in enumerate(rel_docs, 1):
    print(f'| ЗАПРОС {idx}\n{doc.page_content}\n----------\n')

| ЗАПРОС 1
Заголовок: В Москве появится памятник автору "Русского леса"
Содержание: Памятник Леониду Леонову, советскому писателю, автору романа "Русский лес", появится в Москве с одобрения комиссии по монументальному искусству. Об этом сообщает "Интерфакс" со ссылкой на Льва Лавренова, главу комиссии: "Фигура Леонида Леонова значима, и он достоин памятника". Место для монумента пока не определено. Комиссия обсудит этот вопрос с семьей писателя на ближайшем заседании. Леонид Леонов (1899-1994) написал романы "Барсуки", "Вор", "Соть", "Скутаревский", "Дорога на океан", пьесу "Золотая карета", киноповесть "Бегство мистера Мак-Кинли" и другие произведения. Он был лауреатом Ленинской и двух Государственных премий СССР.
Тема: Культура
Теги: Все
Дата: 2008/11/11
----------

| ЗАПРОС 2
Заголовок: Зюганов предложил сделать День космонавтики выходным
Содержание: Лидер КПРФ Геннадий Зюганов предложил сделать День космонавтики 12 апреля выходным, сообщает «Интерфакс». «Я бы учредил этот день выхо

In [11]:
query = 'Прямой эфир с Владимиром Путиным'

rel_docs = retriever.invoke(query)

for idx, doc in enumerate(rel_docs, 1):
    print(f'| ЗАПРОС {idx}\n{doc.page_content}\n----------\n')

| ЗАПРОС 1
Заголовок: В американском шоу спародировали рождественскую встречу Путина и Трампа
Содержание: В новом выпуске комедийного шоу Saturday Night Live показали скетч про встречу избранного президента США Дональда Трампа и российского лидера Владимира Путина в канун Рождества. Ролик был опубликован на YouTube-канале передачи и за сутки собрал почти два миллиона просмотров. По сюжету, Путин в исполнении актера Бэка Беннета пробирается в кабинет к Трампу, сыгранному Алеком Болдуином, с мешком подарков и дарит ему куклу с функцией скрытого наблюдения. После этого к ним присоединяется выдвинутый Трампом на пост госсекретаря бывший глава корпорации ExxonMobil Рекс Тиллерсон (его роль исполнил актер Джон Гудман), который тепло приветствует Путина и обсуждает с ним вопросы нефтедобычи. В конце ноября Трамп раскритиковал один из выпусков Saturday Night Live, в котором его роль также исполнил Алек Болдуин. Избранный президент США назвал шоу «однобоким и предвзятым».
Тема: Интернет и СМИ
Т

In [12]:
query = 'Во сколько лет начинается пенсионный возраст в России?'

rel_docs = retriever.invoke(query)

for idx, doc in enumerate(rel_docs, 1):
    print(f'| ЗАПРОС {idx}\n{doc.page_content}\n----------\n')

| ЗАПРОС 1
Заголовок: В Минтруда разъяснили порядок начисления пенсий для женщин
Содержание: В Министерстве труда и социальной защиты разъяснили новые правила расчета срока выплаты и размера накопительной части пенсии для россиянок. Об этом сообщает РИА Новости со ссылкой на пресс-службу ведомства. Сумма ежемесячных накопительных выплат у них будет на 100 рублей меньше, чем у мужчин. Но общая сумма выплат от этого не изменится. Как поясняют в ведомстве, она будет выплачиваться меньшими частями в течение более длительного срока, так как женщины выходят на пенсию на пять лет раньше мужчин и в среднем на пять лет дольше получают пенсии. «Поэтому, строго говоря, утверждать, что накопительная часть у женщин будет меньше, некорректно», — отметили в министерстве. Ранее газета «Известия» сообщила о новой методике оценки выплат накопительной пенсии, которая будет применяться Пенсионным фондом России (ПФР) и негосударственными пенсионными фондами (НПФ) с 2016 года. В публикации отмечалось, что р

In [13]:
query = 'Что такое ВКонтакте?'

rel_docs = retriever.invoke(query)

for idx, doc in enumerate(rel_docs, 1):
    print(f'| ЗАПРОС {idx}\n{doc.page_content}\n----------\n')

| ЗАПРОС 1
Заголовок: Ющенко заведет микроблог
Содержание: Президент Украины Виктор Ющенко 24 февраля откроет свое официальное представительство в микроблогерском сервисе Twitter, передает УНИАН со ссылкой на слова пресс-секретаря президента Ирины Ванниковой. Главной целью президентского микроблога станет коммуникация главы государства с обществом при помощи удобных современных форм общения. По словам Ванниковой, это не последние изменения в общении Ющенко с обществом. Адрес микроблога - President_UA. Twitter является крупнейшей микроблогерской сетью, объединяя несколько миллионов человек. Запись в Twitter ограничена 140 символами. При использовании политиками и звездами формат Twitter позволяет, например, проводить конференции с пользователями, собирая через микроблог вопросы. В Twitter уже зарегистрированы Барак Обама, Стивен Фрай, Бритни Спирс, Эл Гор, Джон Маккейн и многие другие известные личности.
Тема: Интернет и СМИ
Теги: Все
Дата: 2009/02/24
----------

| ЗАПРОС 2
Заголовок: К